In [1]:
import os
import numpy as np
from scipy.misc import imread, imresize
import tensorflow as tf
import matplotlib.pyplot as plt

# Hyper Paremeter #

### - Image size ###
resize할 image의 height와 width에 관한 hyper parameter

In [2]:
img_height = 64 # 이미지 height 크기
img_width = 64 # 이미지 width의 크기

### - Data ratio ###
전체 데이터 중 학습 데이터(train / validation) 와 테스트 데이터 비율 <br>
기본 값으로 학습 데이터(80%), 테스트 데이터(20%) <br>
학습 데이터 중 (순수 학습 데이터 80% / 평가 데이터 20%) <br>
![1](https://i.imgur.com/jRQ8LZl.png)

In [3]:
train_val_ratio = 0.8 # train 데이터 + validation 데이터의 비율, 나머지는 test 데이터
train_ratio = 0.8 # train 데이터 + validation 데이터에서 tarin 데이터의 비율 , 나머지는 validation 데이터

### Directory & Category Setting ###
기본적으로 jupyter notebook 과 이미지 폴더의 디렉토리 구성이 필요합니다. <br>

jupyter notebook(file) <br>
image(folder) - category1(folder) - image1(file)<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;- image2(file)<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;- ...<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; - category2(folder) - image file 1<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;- image2(file)<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;- ...<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; - ...<br>

In [4]:
tfrecord_train = 'train.tfrecord' # 학습 데이터
tfrecord_val = 'val.tfrecord' # 평가 데이터
tfrecord_test = 'test.tfrecord' # 테스트 데이터
tfrecord_dir = 'tfrecords' #tfrecord 저장할 directory 이름

cur_dir = os.getcwd() #현재 디렉토리 path 출력
image_dir = os.path.join(cur_dir,'image') # 이미지 경로 설정
valid_exts = ['.jpg', '.gif', '.png', '.jpeg'] # 허용 가능한 확장자 설정
print('%d categories in %s' % (len(os.listdir(image_dir)), image_dir)) #image 폴더 내 카테고리 출력 (Dog, Cat)
categories = sorted(os.listdir(image_dir)) # Image_dir 하위 Directory를 카테고리 별로 분류함, 카테고리 이름 
num_categ = len(categories) # category 갯수 
print(categories) # category 출력

2 categories in D:\Code\Python\Deep_Learning\Trial & Error\Cat_Dog_binary_classification\image
['cat', 'dog']


### Extract & Organize image list ###

In [5]:
imgnames = [] # image name list
labels = [] # label list

for label, category in enumerate(categories): # category 별로 label을 붙입니다.
    filelist = os.listdir(os.path.join(image_dir, category)) # 해당 디렉터리에 있는 파일들의 리스트를 구할 수 있다.
    imglist = []
    for f in filelist:
        img_path = os.path.join(image_dir, category, f)
        
        # 1) 이미지 확장자 아닌 파일 제거
        ext = os.path.splitext(f)[-1] # 파일 명에서 확장자만 가져온다.
        if ext.lower() not in valid_exts: # 현재 파일의 확장자가 올바른 이미지 확장자 인지 판단합니다.
            os.remove(os.path.join(img_path)) # 기본 확장지 이외의 데이터는 삭제 후 pass            
            continue
        
        # 2) 정상 이미지( shape = 3 & channel = 3 (R,G,B))가 아닌 경우 제거        
        image = np.array(imread(img_path)) # image를 받습니다.
        if len(image.shape) != 3 or image.shape[2] !=3: # (height, width, channel) 형태의 shape이 3이고 channel은 (R, G, B) 3개를 가져야 합니다.
            os.remove(os.path.join(img_path))
            continue
            
        imglist.append(f) # 올바른 이미지는 imglist에 추가합니다.
        
    imgnames += imglist # imglist에 추가된 이미지만 최종 imgnames에 추가합니다.
    labels += [label] * len(imglist) # imglist에 추가된 갯수만큼 label을 만듭니다. label은 카테고리별로 0, 1, 2, ...로 증가합니다.    
    print("%d %s images are found" % (len(imglist), category))

2395 cat images are found
4988 dog images are found


### Split data(Train/Validation/Test) ###

In [6]:
numfiles = len(labels) # 전체 이미지 갯수
train_val_numfiles = int(train_val_ratio * numfiles)
train_numfiles = int(train_ratio * train_val_numfiles)

idxrand = np.random.permutation(numfiles) #전체 이미지 갯수 : [0, numfiles) 만큼 순서 랜덤으로 섞음
idxtrain = idxrand[:train_numfiles] # train 데이터 인덱스
idxval = idxrand[train_numfiles:train_val_numfiles] # validation 데이터 인덱스
idxtest = idxrand[train_val_numfiles:] # test 데이터 인덱스

print('#train data : ', len(idxtrain))
print('#validation data : ', len(idxval))
print('#test data : ', len(idxtest))

#train data :  4724
#validation data :  1182
#test data :  1477


### TFRecord 생성 ###
- Train data / Validation data / Test data 각각의 TFRecordWriter를 통하여 학습 데이터 생성 

In [7]:
def _int64_feature(value):
    return tf.train.Feature(int64_list = tf.train.Int64List(value=[value]))

def _bytes_feature(value):
    return tf.train.Feature(bytes_list = tf.train.BytesList(value=[value]))

In [9]:
writer_train = tf.python_io.TFRecordWriter(os.path.join(cur_dir, tfrecord_dir, tfrecord_train))
writer_val = tf.python_io.TFRecordWriter(os.path.join(cur_dir, tfrecord_dir, tfrecord_val))
writer_test = tf.python_io.TFRecordWriter(os.path.join(cur_dir, tfrecord_dir, tfrecord_test))

for idx in idxtrain:
    label = labels[idx]
    img_path = os.path.join(image_dir, categories[label], imgnames[idx])
    image = np.array(imread(img_path))            
    
    image = imresize(image, [img_height, img_width])
    image_str = image.tostring()
    
    example = tf.train.Example(features=tf.train.Features(feature={
        'image': _bytes_feature(image_str),
        'label': _int64_feature(label)
    }))
    writer_train.write(example.SerializeToString())
    
for idx in idxval:
    label = labels[idx]
    img_path = os.path.join(image_dir, categories[label], imgnames[idx])
    image = np.array(imread(img_path))            
    
    image = imresize(image, [img_height, img_width])
    image_str = image.tostring()
    
    example = tf.train.Example(features=tf.train.Features(feature={
        'image': _bytes_feature(image_str),
        'label': _int64_feature(label)
    }))
    writer_val.write(example.SerializeToString())    
    
for idx in idxtest:
    label = labels[idx]
    img_path = os.path.join(image_dir, categories[label], imgnames[idx])
    image = np.array(imread(img_path))   
    
    image = imresize(image, [img_height, img_width])
    image_str = image.tostring()
    
    example = tf.train.Example(features=tf.train.Features(feature={
        'image': _bytes_feature(image_str),
        'label': _int64_feature(label)
    }))
    writer_test.write(example.SerializeToString())

writer_train.close()
writer_val.close()
writer_test.close()